In [55]:
import torch
import triton
import triton.language as tl
import time

In [56]:
def is_cuda():
    return triton.runtime.driver.active.get_current_target().backend == "cuda"

In [57]:
def is_hip_mi200():
    target = triton.runtime.driver.active.get_current_target()
    return target.backend == 'hip' and target.arch == 'gfx90a'

In [58]:
"""
PA2 Part 1: MatMul+Relu+Add Fused Optimization.
The kernel uses several optimization techniques:

    1. Shared memory tiling.
    2. Register tiling.
    3. Cooperative fetching.
    4. Operator Fusion
    5. Write cache / epilogue fusion.

Fill in the missing parts (marked with TODO).
"""

# -----------------------------------------------------------------------------
# Tiling parameters - You will need to change these to achieve better results.
# -----------------------------------------------------------------------------
BLOCK_M = 128  # Tile size in the M dimension.
BLOCK_N = 256 # Tile size in the N dimension.
BLOCK_K = 16 # Tile size in the K dimension.


# -----------------------------------------------------------------------------
# Triton Kernel: Matrix Multiplication + ReLU + Add
#
# The kernel uses:
#   Step 1: Tile assignment (each kernel computes a tile of C)
#   Step 2: Shared memory tiling + Cooperative Fetching: Load tiles of A and B.
#   Step 3: Register tiling: Use a register accumulator.
#   Step 4: Add and ReLU fusion
#   Step 5: Write cache/Epilogue: Write the final tile back to global memory.
# -----------------------------------------------------------------------------
@triton.jit
def matmul_add_relu_kernel_fp16(
    a_ptr, b_ptr, c_ptr, d_ptr,
    M: tl.constexpr, N: tl.constexpr, K: tl.constexpr,
    stride_am: tl.constexpr, stride_ak: tl.constexpr,
    stride_bk: tl.constexpr, stride_bn: tl.constexpr,
    stride_cm: tl.constexpr, stride_cn: tl.constexpr,
    stride_dm: tl.constexpr, stride_dn: tl.constexpr,
    BLOCK_M: tl.constexpr, BLOCK_N: tl.constexpr, BLOCK_K: tl.constexpr,
):
    # -------------------------------------------------------------------------
    # Step 1: Tile: Assignment
    #
    # Each kernel instance is mapped to a tile in the output matrix C.
    # Compute the starting indices (m_start, n_start) for this tile.
    # -------------------------------------------------------------------------
    # Compute the tile indices using program_id(0) for M and program_id(1) for N.

    # computing program ids to determine which block of output matrix (say C) to compute.
    pid_m = tl.program_id(0) # row block
    pid_n = tl.program_id(1) # column block

    # -------------------------------------------------------------------------
    # Step 2: Register Tiling
    # -------------------------------------------------------------------------
    # Initialize the accumulator "acc" with zeros (dtype: float16).

    acc = tl.zeros((BLOCK_M, BLOCK_N), dtype=tl.float16)

    # -------------------------------------------------------------------------
    # Step 3: Shared Memory Tiling & Cooperative Fetching.
    # Compute pointers to the sub-tiles of A and B that are needed to compute
    # the current C tile. The offsets here serve to load BLOCK_SIZE_M x BLOCK_SIZE_K
    # and BLOCK_SIZE_K x BLOCK_SIZE_N blocks from A and B respectively.
    # -------------------------------------------------------------------------

    # computing row and column offsets for the current block
    offs_am = pid_m * BLOCK_M + tl.arange(0, BLOCK_M)
    offs_bn = pid_n * BLOCK_N + tl.arange(0, BLOCK_N)
    offs_k = tl.arange(0, BLOCK_K)

    # creating pointers for loading A, B
    a_ptrs = a_ptr + (offs_am[:, None] * stride_am + offs_k[None, :] * stride_ak)
    b_ptrs = b_ptr + (offs_k[:, None] * stride_bk + offs_bn[None, :] * stride_bn)

    for k in range(0, tl.cdiv(K, BLOCK_K)):

        # loading A and B blocks with masking
        a = tl.load(a_ptrs, mask=offs_k[None, :] < K - k * BLOCK_K, other=0.0)
        b = tl.load(b_ptrs, mask=offs_k[:, None] < K - k * BLOCK_K, other=0.0)

        # performing matrix multiplication on the current block
        acc += tl.dot(a, b, out_dtype=tl.float16)

        # moving pointers to the next block along K
        a_ptrs += BLOCK_K * stride_ak
        b_ptrs += BLOCK_K * stride_bk

    # -------------------------------------------------------------------------
    # Step 4: Apply ReLU and Add C to the accumulator
    # -------------------------------------------------------------------------

    # creating pointers for loading matrix C
    c_ptrs = c_ptr + (offs_am[:, None] * stride_cm + offs_bn[None, :] * stride_cn)

    # loading C matrix elements
    c = tl.load(c_ptrs, mask=(offs_am[:, None] < M) & (offs_bn[None, :] < N), other=0.0)

    # adding C and applying ReLU
    acc += c
    acc = tl.maximum(acc, 0)

    # -------------------------------------------------------------------------
    # Step 5: Write Cache / Epilogue Fusion: Write the computed tile to D.
    # -------------------------------------------------------------------------

    # creating pointers for storing the computed block to D
    d_ptrs = d_ptr + (offs_am[:, None] * stride_dm + offs_bn[None, :] * stride_dn)

    # storing the computed block to D
    tl.store(d_ptrs, acc, mask=(offs_am[:, None] < M) & (offs_bn[None, :] < N))

In [59]:
def matmul_add_relu_fp16(a: torch.Tensor, b: torch.Tensor, c: torch.Tensor) -> torch.Tensor:
    """
    Computes Output = ReLU(A @ B + C) using fp16 precision for maximum throughput.
    """
    M, K = a.shape
    K2, N = b.shape
    assert K == K2, "Incompatible dimensions"

    d = torch.empty((M, N), device=a.device, dtype=torch.float16)
    # Create launch grid
    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(N, BLOCK_N))

    matmul_add_relu_kernel_fp16[grid](
        a, b, c, d,
        M, N, K,
        a.stride(0), a.stride(1),
        b.stride(0), b.stride(1),
        c.stride(0), c.stride(1),
        d.stride(0), d.stride(1),
        BLOCK_M=BLOCK_M, BLOCK_N=BLOCK_N, BLOCK_K=BLOCK_K
    )
    return d

In [60]:
# Reference implementation using PyTorch
def reference_matmul_add_relu(A, B, C):
    result = torch.matmul(A, B).add(C).relu_()
    return result

In [61]:
# -----------------------------------------------------------------------------
# Accuracy Tests
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    torch.manual_seed(0)
    a = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    b = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    c = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    triton_output = matmul_add_relu_fp16(a, b, c)
    torch_output = reference_matmul_add_relu(a, b, c)
    print(f"triton_output_with_fp16_inputs={triton_output}")
    print(f"torch_output_with_fp16_inputs={torch_output}")
    rtol = 1e-2 if is_hip_mi200() else 0.032
    if torch.allclose(triton_output, torch_output, atol=0.15, rtol=rtol):
        print("✅ Triton and Torch match")
    else:
        diff = triton_output - torch_output
        abs_diff = torch.abs(diff)
        max_abs_diff = torch.max(abs_diff)
        print(f"❌ Triton and Torch differ: {max_abs_diff=}")

triton_output_with_fp16_inputs=tensor([[ 0.0000,  6.1250,  0.0000,  ..., 10.0625,  0.0000,  0.0000],
        [ 7.9102, 15.6328, 26.6094,  ..., 11.4609,  5.3750, 18.6250],
        [ 2.7246,  0.0000,  0.0000,  ...,  0.0000, 26.0781,  0.0000],
        ...,
        [ 0.4448, 75.1875,  0.0000,  ..., 26.2812,  0.0000,  0.0000],
        [ 6.9492,  1.1230,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [27.6094, 26.9531, 22.9219,  ..., 13.5391,  6.0508, 21.6250]],
       device='cuda:0', dtype=torch.float16)
torch_output_with_fp16_inputs=tensor([[ 0.0000,  6.1289,  0.0000,  ..., 10.0391,  0.0000,  0.0000],
        [ 7.9102, 15.6328, 26.6250,  ..., 11.4531,  5.3945, 18.6562],
        [ 2.7266,  0.0000,  0.0000,  ...,  0.0000, 26.1250,  0.0000],
        ...,
        [ 0.4316, 75.2500,  0.0000,  ..., 26.2812,  0.0000,  0.0000],
        [ 6.9570,  1.1260,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [27.6406, 26.9531, 22.9375,  ..., 13.5625,  6.0391, 21.6406]],
       device='cuda:0', dt

In [66]:
# -----------------------------------------------------------------------------
# Performance Benchmark
# IMPORTANT: DO NOT CHANGE THIS CODE.
# THIS IS THE EXACT CODE THAT WILL BE USED TO GRADE YOUR IMPLEMENTATION.
# ANY CHANGES TO THIS CODE (INCLUDING DIMENSIONS, REPEATS, etc.)
# WILL CAUSE YOU TO HAVE DIFFERENT SPEEDUP RESULTS.
# -----------------------------------------------------------------------------
M = 2048
K = 2048
N = 2048

# KEEP THESE MATRICES IN FP16. FP32 WILL NOT PROVIDE ACCURATE RESULTS
A = torch.randn((M, K), device="cuda", dtype=torch.float16)
B = torch.randn((K, N), device="cuda", dtype=torch.float16)
C = torch.randn((M, N), device="cuda", dtype=torch.float16)

# warmup
_ = matmul_add_relu_fp16(A, B, C)
_ = reference_matmul_add_relu(A, B, C)

REPEATS = 5000

# time your implementation
print("Triton implementation")
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(REPEATS):
    _ = matmul_add_relu_fp16(A, B, C)
torch.cuda.synchronize()
triton_time = (time.perf_counter() - start) / REPEATS

# time pytorch
print("PyTorch implementation")
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(REPEATS):
    _ = reference_matmul_add_relu(A, B, C)
torch.cuda.synchronize()
torch_time = (time.perf_counter() - start) / REPEATS

print(f"Performance comparison for matrix multiplication ({M}x{K} @ {K}x{N}):")
print(f"Triton implementation: {triton_time*1000:.2f} ms")
print(f"PyTorch implementation: {torch_time*1000:.2f} ms")

print(f"\nSpeedup of Triton vs PyTorch: {torch_time/triton_time:.2f}x")

Triton implementation
PyTorch implementation
Performance comparison for matrix multiplication (2048x2048 @ 2048x2048):
Triton implementation: 0.74 ms
PyTorch implementation: 1.01 ms

Speedup of Triton vs PyTorch: 1.36x


In [68]:
# Write your grid search here.

# Grid search for optimal block sizes
def benchmark_matmul(A, B, C, block_m, block_n, block_k, num_repeats=REPEATS):
    global BLOCK_M, BLOCK_N, BLOCK_K
    BLOCK_M = block_m
    BLOCK_N = block_n
    BLOCK_K = block_k

    # Warmup
    _ = matmul_add_relu_fp16(A, B, C)
    torch.cuda.synchronize()

    # Time Triton implementation
    start = time.perf_counter()
    for _ in range(num_repeats):
        _ = matmul_add_relu_fp16(A, B, C)
    torch.cuda.synchronize()
    triton_time = (time.perf_counter() - start) / num_repeats

    return triton_time

# Generate test matrices
M = N = K = 2048
A = torch.randn((M, K), device="cuda", dtype=torch.float16)
B = torch.randn((K, N), device="cuda", dtype=torch.float16)
C = torch.randn((M, N), device="cuda", dtype=torch.float16)

# Get PyTorch baseline
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(REPEATS):
    _ = reference_matmul_add_relu(A, B, C)
torch.cuda.synchronize()
torch_time = (time.perf_counter() - start) / REPEATS

# Define search space (powers of 2)
block_m_sizes = [32, 64, 128, 256]
block_n_sizes = [32, 64, 128, 256]
block_k_sizes = [16, 32, 64]

best_speedup = 0
best_params = None

print("Starting grid search...")
print(f"PyTorch baseline: {torch_time*1000:.2f} ms")
print("\nTesting configurations:")

for block_m in block_m_sizes:
    for block_n in block_n_sizes:
        for block_k in block_k_sizes:
            try:
                triton_time = benchmark_matmul(A, B, C, block_m, block_n, block_k)
                speedup = torch_time / triton_time

                print(f"M={block_m}, N={block_n}, K={block_k}: {triton_time*1000:.2f} ms (speedup: {speedup:.2f}x)")

                if speedup > best_speedup:
                    best_speedup = speedup
                    best_params = (block_m, block_n, block_k)
            except Exception as e:
                print(f"Error with M={block_m}, N={block_n}, K={block_k}: {str(e)}")
                continue

print(f"\nBest configuration found:")
print(f"BLOCK_M = {best_params[0]}")
print(f"BLOCK_N = {best_params[1]}")
print(f"BLOCK_K = {best_params[2]}")
print(f"Speedup: {best_speedup:.2f}x")

# Set the best parameters
BLOCK_M = best_params[0]
BLOCK_N = best_params[1]
BLOCK_K = best_params[2]


Starting grid search...
PyTorch baseline: 0.95 ms

Testing configurations:
M=32, N=32, K=16: 4.32 ms (speedup: 0.22x)
M=32, N=32, K=32: 4.20 ms (speedup: 0.23x)
M=32, N=32, K=64: 3.42 ms (speedup: 0.28x)
M=32, N=64, K=16: 2.62 ms (speedup: 0.36x)
M=32, N=64, K=32: 2.51 ms (speedup: 0.38x)
M=32, N=64, K=64: 2.19 ms (speedup: 0.43x)
M=32, N=128, K=16: 1.82 ms (speedup: 0.52x)
M=32, N=128, K=32: 1.72 ms (speedup: 0.55x)
M=32, N=128, K=64: 1.49 ms (speedup: 0.64x)
M=32, N=256, K=16: 1.32 ms (speedup: 0.72x)
M=32, N=256, K=32: 1.35 ms (speedup: 0.70x)
M=32, N=256, K=64: 1.51 ms (speedup: 0.63x)
M=64, N=32, K=16: 3.40 ms (speedup: 0.28x)
M=64, N=32, K=32: 3.36 ms (speedup: 0.28x)
M=64, N=32, K=64: 2.70 ms (speedup: 0.35x)
M=64, N=64, K=16: 2.10 ms (speedup: 0.45x)
M=64, N=64, K=32: 1.97 ms (speedup: 0.48x)
M=64, N=64, K=64: 1.68 ms (speedup: 0.57x)
M=64, N=128, K=16: 1.23 ms (speedup: 0.78x)
M=64, N=128, K=32: 1.29 ms (speedup: 0.74x)
M=64, N=128, K=64: 1.08 ms (speedup: 0.88x)
M=64, N=256, 